In [2]:
library(ggplot2)
library(data.table)
library(cowplot)

setwd('/data6/Angelo/alexcc/AngeloStrainsPaper/manuscript_materials/soil_popgen/notebooks')

Registered S3 method overwritten by 'dplyr':
  method               from  
  as.data.frame.tbl_df tibble

********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************



In [21]:
sizes = read.csv('../data_tables/genome_sizes.txt', header=FALSE, sep="\t")

In [39]:
## get nucleotide diversities
results <- data.frame('count' = numeric(),'pi'=numeric(),'coverage'=numeric(),
                      stringsAsFactors = FALSE)
for (file in Sys.glob('../data/sample_snps/*.tsv')){
    genome = gsub(gsub(gsub(file, pattern = '.aa-SNVs.tsv', replacement = ''), 
     pattern='../data/sample_snps/', replacement = ''), pattern=':.*', replacement='')
    size = sizes[which(sizes$V1 == genome),'V2']
    d = fread(file)
    count = nrow(d) / size * 1000000
    d2 = fread(gsub(gsub(file, pattern = '.aa-SNVs.tsv', replacement = '.genes.tsv'), 
               pattern='sample_snps/', replacement='nucleotide_diversity/'))
    avg = mean(d2$pi)
    coverage = mean(d2$coverage)
    results[nrow(results) + 1,] = list(count, avg, coverage)
    
}

In [40]:
d = results

p1 <- qplot(x = d$count, y= d$coverage, geom = 'point') + theme_minimal() + xlab("SNPs / Mbp") + ylab("Coverage")
p2 <- qplot(x = d$pi, y= d$coverage, geom = 'point') + theme_minimal() + xlab("Nucleotide Diversity") + ylab("Coverage")

p3 <- qplot(x = d$count, y= d$pi, geom = 'point') + theme_minimal() + xlab("SNPs") + ylab("Nucleotide Diversity")
p4 <- qplot(x = d$count, y= d$pi*d$coverage, geom = 'point') + theme_minimal() + xlab("SNPs / Mbp") + ylab("Nucleotide Diversity*coverage")


In [42]:
pdf(file = 'statistics.pdf')
plot_grid(p1,p2,p3,p4)
dev.off()

png 
  2

In [3]:
## Read in nucleotide diversity data for 0.96
files = Sys.glob("../data/cutoff_comparison/0.96/*.tsv")

i = 0;
for (f in files){
    d2 = read.csv(f, sep='\t')
#     print(f)
    if (i > 0){
        d96 = rbind(d96,d2)
    }
    else {
        d96 = d2
        print("step 1")
    }
    i = i + 1
}

[1] "step 1"


In [4]:
## Read in nucleotide diversity data for 0.98
files = Sys.glob("../data/cutoff_comparison/0.98/*.tsv")

i = 0;
for (f in files){
    d2 = read.csv(f, sep='\t')
#     print(f)
    if (i > 0){
        d98 = rbind(d98,d2)
    }
    else {
        d98 = d2
        print("step 1")
    }
    i = i + 1
}

[1] "step 1"


In [5]:
d98 = d98[,c('gene', 'pi')]
d96 = d96[,c('gene', 'pi')]

In [6]:
combined_d = merge(d98, d96, by='gene')

In [52]:
m <- lm(log(pi.y+0.00001, base=10) ~ log(pi.x+0.00001, base=10), combined_d)
summary(m)


Call:
lm(formula = log(pi.y + 1e-05, base = 10) ~ log(pi.x + 1e-05, 
    base = 10), data = combined_d)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.60799 -0.05920 -0.02612  0.03253  2.89157 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  0.1397748  0.0022294    62.7   <2e-16 ***
log(pi.x + 1e-05, base = 10) 1.0237894  0.0009587  1067.9   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.08711 on 64210 degrees of freedom
Multiple R-squared:  0.9467,	Adjusted R-squared:  0.9467 
F-statistic: 1.14e+06 on 1 and 64210 DF,  p-value: < 2.2e-16


In [34]:
library(viridis)

Loading required package: viridisLite


[1] 0.9969486

In [64]:
pdf(file = 'figs1_2.pdf')
ggplot(data=combined_d, aes(log(pi.x, base = 10), log(pi.y, base= 10)))  + geom_hex() + geom_smooth(method='lm', size=2) + 
xlab("Log(98% read ANI gene nucleotide diversity)") + 
    ylab("Log(96% read ANI gene nucleotide diversity)") + scale_fill_viridis(alpha = 0.7) + theme_minimal()
dev.off()

Warning message:
“Removed 25 rows containing non-finite values (stat_binhex).”Warning message:
“Removed 25 rows containing non-finite values (stat_smooth).”

png 
  2

In [1]:
cor.test(x = combined_d$pi.y, y= combined_d$pi.x)

ERROR: Error in cor.test(x = combined_d$pi.y, y = combined_d$pi.x): object 'combined_d' not found


[1] "0.91"